In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
import random
import os
import re

# 크롤링 결과를 저장할 파일 이름
OUTPUT_FILE = ""

def setup_driver():
    """셀레니움 웹드라이버 설정"""
    chrome_options = Options()
    
    # 디버깅을 위해 헤드리스 모드 비활성화 (실제 실행 시 주석 해제)
    chrome_options.add_argument("--headless")
    
    # 봇 감지 회피 설정
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    
    # 일반적인 사용자처럼 보이는 UserAgent 설정
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    chrome_options.add_argument("--window-size=1920,1080")
    
    # 웹드라이버 생성
    driver = webdriver.Chrome(options=chrome_options)
    
    # 웹드라이버 감지 회피를 위한 추가 설정
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    return driver

def random_sleep(min_seconds=1, max_seconds=3):
    """무작위 시간 동안 대기"""
    time.sleep(random.uniform(min_seconds, max_seconds))

def extract_product_name_and_content(full_text, debug=False):
    """전체 텍스트에서 제품명과 내용을 추출하는 함수"""
    product_name = None
    content = None
    rating = None
    
    # 텍스트가 None인 경우 처리
    if full_text is None:
        return product_name, content, rating
    
    # 1. [CODE] 패턴을 포함한 제품명 추출
    # 대괄호 코드 패턴 [SW...], [LSW...] 등과 그 앞 뒤 텍스트를 제품명으로 추출
    code_pattern = r'(?:[\w\s가-힣\(\)]+)?\s*\[(?:SW|LSW|KPC|PC|ATA|ASA|A5A|A3A|SC|NES|DMC|LGNS|PCS)[\w\d-]+\]\s*(?:[\w\s가-힣\(\)]+)?'
    code_matches = re.finditer(code_pattern, full_text)
    
    for match in code_matches:
        # 제품명 후보 찾기
        candidate = match.group(0).strip()
        
        # 제품명에 리뷰나 별점 키워드가 포함되어 있으면 제외
        if "리뷰" not in candidate.lower() and "별점" not in candidate.lower() and "님의 리뷰입니다" not in candidate:
            product_name = candidate
            
            # 제품명 이후의 텍스트를 내용으로 간주
            product_end_idx = match.end()
            remaining_text = full_text[product_end_idx:].strip()
            
            # 별점/리뷰 패턴 찾기 (x.x 리뷰)
            rating_match = re.search(r'(\d+\.\d+)\s*리뷰', remaining_text)
            if rating_match:
                rating = float(rating_match.group(1))
                
                # 별점 패턴 이후의 텍스트에서 여러 줄바꿈 다음의 내용을 추출
                rating_end_idx = remaining_text.find(rating_match.group(0)) + len(rating_match.group(0))
                content_text = remaining_text[rating_end_idx:].strip()
                
                # 여러 줄바꿈 패턴 이후의 텍스트를 실제 내용으로 간주
                multiple_newline_match = re.search(r'\n\n+', content_text)
                if multiple_newline_match:
                    newline_end_idx = content_text.find(multiple_newline_match.group(0)) + len(multiple_newline_match.group(0))
                    content = content_text[newline_end_idx:].strip()
                else:
                    content = content_text
            else:
                # 별점 패턴이 없으면 제품명 이후의 텍스트를 내용으로 처리
                content = remaining_text
            
            break
    
    # 제품명을 찾지 못한 경우, 내용에 [CODE] 패턴이 있는지 확인
    if not product_name and full_text:
        # [CODE] 패턴만 찾기
        simple_code_match = re.search(r'\[((?:SW|LSW|KPC|PC|ATA|ASA|A5A|A3A|SC|NES|DMC|LGNS|PCS)[\w\d-]+)\]', full_text)
        if simple_code_match:
            # 대괄호 코드 주변 텍스트를 제품명으로 추출
            code_start_idx = full_text.find(simple_code_match.group(0))
            
            # 코드의 앞/뒤 50자를 포함하여 제품명 추출 시도
            context_start = max(0, code_start_idx - 50)
            context_end = min(len(full_text), code_start_idx + len(simple_code_match.group(0)) + 50)
            context = full_text[context_start:context_end]
            
            # 제품명 경계 식별 (줄바꿈, 별점, 특수 패턴 등)
            name_start = 0
            if context_start > 0:
                # 앞쪽 경계 찾기 (줄바꿈, 특수 패턴)
                for boundary in ['\n', '리뷰입니다', '별점:', '★']:
                    boundary_idx = context[:code_start_idx-context_start].rfind(boundary)
                    if boundary_idx >= 0:
                        name_start = boundary_idx + len(boundary)
                        break
            
            name_end = len(context)
            # 뒤쪽 경계 찾기 (별점, 리뷰, 줄바꿈)
            for boundary in ['리뷰', '별점:', '★', '\n']:
                boundary_idx = context[code_start_idx-context_start+len(simple_code_match.group(0)):].find(boundary)
                if boundary_idx >= 0:
                    name_end = code_start_idx-context_start+len(simple_code_match.group(0)) + boundary_idx
                    break
            
            product_name = context[name_start:name_end].strip()
            
            # 제품명 이후 텍스트를 내용으로 간주
            content = full_text[context_start + name_end:].strip()
    
    # 제품명/내용 추출에 실패한 경우 기본값 설정
    if not product_name:
        product_name = None
    if not content:
        content = full_text
    
    return product_name, content, rating

def extract_writer_info(full_text):
    """작성자 정보 추출 함수"""
    writer = None
    
    # 텍스트가 None인 경우 처리
    if full_text is None:
        return writer
    
    # 네이버페이 구매자 확인
    naverpay_match = re.search(r'네이버\s*페이\s*구매자', full_text, re.IGNORECASE)
    if naverpay_match:
        return "네이버페이 구매자"
    
    # 여러 작성자 패턴 시도
    writer_patterns = [
        r'(\w+\*+)님의 리뷰입니다',          # 일반적인 패턴: xxx*님의 리뷰입니다
        r'(\d{7,9}\*+)님의 리뷰입니다',      # 숫자 아이디 패턴
        r'(\w+)[이가] 작성한 리뷰',           # 다른 형식
        r'작성자[:\s]*(\w+\*+)',             # 작성자: xxx* 형식
        r'(\w+\*+)의 구매 후기'              # xxx*의 구매 후기 형식
    ]
    
    for pattern in writer_patterns:
        writer_match = re.search(pattern, full_text)
        if writer_match:
            writer = writer_match.group(1)
            break
    
    # 이메일 형식의 아이디 처리
    if not writer:
        email_match = re.search(r'([a-zA-Z0-9_\.\-]+\*+@[a-zA-Z0-9_\.\-]+)', full_text)
        if email_match:
            writer = email_match.group(1)
    
    # 아주 좋아요 패턴의 리뷰자
    if not writer and "아주 좋아요" in full_text:
        writer = "아주 좋아요"
    
    return writer

def extract_review_data(element_text, debug=False):
    """리뷰 텍스트에서 모든 정보를 추출하는 함수"""
    review_data = {
        "product_name": None,
        "rating": None,
        "writer": None,
        "date": None,
        "content": None,
        "height": None,           # 키
        "weight": None,           # 체중
        "usual_size": None,       # 평소사이즈
        "member_level": None,     # 회원 등급
        "size": None,             # 사이즈
    }
    
    # 텍스트가 None인 경우 처리
    if element_text is None:
        return review_data
    
    # 디버깅 목적으로 원본 텍스트 저장
    if debug:
        print(f"원본 텍스트: {element_text[:300]}...")
    
    # 1. 제품명, 내용, 별점 추출
    product_name, content, rating = extract_product_name_and_content(element_text, debug)
    review_data["product_name"] = product_name
    review_data["content"] = content
    review_data["content"] = extract_clean_review_content(review_data["content"])
    review_data["rating"] = rating
    
    # 2. 작성자 정보 추출
    review_data["writer"] = extract_writer_info(element_text)
    
    # 3. 날짜 추출 (YYYY.MM.DD 형식)
    date_match = re.search(r'(\d{4}\.\d{1,2}\.\d{1,2})', element_text)
    if date_match:
        review_data["date"] = date_match.group(1)
    
    # 4. 별점이 추출되지 않은 경우 다른 패턴 시도
    if not review_data["rating"]:
        # 별점 추출 (다양한 패턴)
        rating_patterns = [
            r'별점[:=\s]*(\d+(\.\d+)?)',    # 별점: 4.5 형식
            r'(\d+(\.\d+)?)\s*점',           # 4.5점 형식
            r'(\d+(\.\d+)?)\s*리뷰',         # 4.5 리뷰 형식
            r'([0-9.]+)\s*별점'              # 4.5 별점 형식
        ]
        
        for pattern in rating_patterns:
            rating_match = re.search(pattern, element_text)
            if rating_match:
                try:
                    review_data["rating"] = float(rating_match.group(1))
                    break
                except:
                    pass
        
        # 별 문자(★) 개수로 별점 추출
        if not review_data["rating"] and '★' in element_text:
            review_data["rating"] = float(element_text.count('★'))
    
    # 5. 사용자 정보 추출 (키, 체중, 사이즈 등)
    # 네이버페이 구매자인 경우 사용자 정보 없음 처리
    writer = review_data["writer"]
    if writer is not None and "네이버페이 구매자" in writer:
        pass  # 사용자 정보 추출 건너뛰기
    else:
        # 텍스트를 줄 단위로 분리
        lines = element_text.split('\n')
        
        for line in lines:
            # 키 정보 추출
            if not review_data["height"]:
                height_match = re.search(r'키[\s:]*(\d+)cm', line, re.IGNORECASE)
                if height_match:
                    review_data["height"] = height_match.group(1)
            
            # 체중 정보 추출
            if not review_data["weight"]:
                weight_match = re.search(r'체중[\s:]*([\d]+(?:\s*~\s*[\d]+)?)kg', line, re.IGNORECASE)
                if weight_match:
                    review_data["weight"] = weight_match.group(1).replace(' ', '')
            
            # 평소 사이즈 추출
            if not review_data["usual_size"]:
                usual_size_match = re.search(r'평소\s*사이즈[\s:]*([\w\d]+)', line, re.IGNORECASE)
                if usual_size_match:
                    review_data["usual_size"] = usual_size_match.group(1)
            
            # 회원 등급 추출
            if not review_data["member_level"]:
                member_level_match = re.search(r'회원\s*등급[\s:]*([\w]+)', line, re.IGNORECASE)
                if member_level_match:
                    review_data["member_level"] = member_level_match.group(1)
            
            # 선택한 사이즈 추출
            if not review_data["size"]:
                if "사이즈" in line and "평소" not in line and "발" not in line:
                    size_match = re.search(r'사이즈[\s:]*((?:[A-Z]+\s?[A-Z]*)|\d{2})', line)
                    if size_match:
                        review_data["size"] = size_match.group(1).replace(" ", "")
    
    # 6. 데이터 최종 정제
    # 내용에서 불필요한 텍스트 제거
    if review_data["content"]:
        # "NEW\n" 같은 패턴 제거
        review_data["content"] = re.sub(r'^NEW\s*\n', '', review_data["content"])
        review_data["content"] = re.sub(r'^NEW\s+', '', review_data["content"])
        
        # 네이버페이 구매자 정보 제거
        if "네이버페이 구매자" in review_data["content"]:
            review_data["content"] = re.sub(r'네이버페이 구매자[^\n]*', '', review_data["content"]).strip()
        
        # (YYYY-MM-DD HH:MM:SS) 같은 타임스탬프 패턴 제거
        review_data["content"] = re.sub(r'\(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}[^\)]*\)', '', review_data["content"]).strip()
        
        # 작성자 정보 제거
        if review_data["writer"] and review_data["writer"] in review_data["content"]:
            review_data["content"] = review_data["content"].replace(review_data["writer"], "").strip()
        
        # "님의 리뷰입니다" 패턴 제거
        review_pattern = re.search(r'님의 리뷰입니다\.?', review_data["content"])
        if review_pattern:
            review_data["content"] = review_data["content"].replace(review_pattern.group(0), "").strip()
        
        # 별점 패턴 제거
        if review_data["rating"]:
            rating_text = f"{review_data['rating']} 리뷰"
            if rating_text in review_data["content"]:
                review_data["content"] = review_data["content"].replace(rating_text, "").strip()
        
        # 날짜 제거
        if review_data["date"] and review_data["date"] in review_data["content"]:
            review_data["content"] = review_data["content"].replace(review_data["date"], "").strip()
        
        # 여러 줄바꿈 제거
        review_data["content"] = re.sub(r'\n{2,}', ' ', review_data["content"]).strip()
        
        # 다양한 별점 키워드 제거
        rating_keywords = ["별점:", "★★★★★", "★★★★", "★★★", "★★", "★"]
        for keyword in rating_keywords:
            if keyword in review_data["content"]:
                review_data["content"] = review_data["content"].replace(keyword, "").strip()
        
        # # "발 쏘았습니다---" 같은 패턴만 있으면 내용이 부족한 것으로 판단
        # if re.match(r'^\s*발\s*쏘았습니다-+\s*$', review_data["content"]):
        #     review_data["content"] = "발 쏘았습니다."
        
        # # 너무 짧거나 의미 없는 내용 제거
        # if review_data["content"] and len(review_data["content"]) < 3:
        #     review_data["content"] = ""
    
    return review_data

def process_review_container(container, debug=False):
    """리뷰 컨테이너에서 모든 정보 추출하는 함수"""
    # 컨테이너의 전체 텍스트 가져오기
    try:
        full_text = container.text
        if debug:
            print(f"컨테이너 텍스트: {full_text[:100]}...")
            if debug:
                print("==== 리뷰 텍스트 미리보기 ====")
                print(full_text[:300])

            # 이미지 디버깅용 print 추가
            images = container.find_elements(By.TAG_NAME, 'img')
            print(f"[디버깅] 이미지 개수: {len(images)}")
            for img in images:
                src = img.get_attribute("src")
                print(f"→ 이미지 src: {src}")
       
        # 직접 상품명 요소 찾기 시도
        product_name = None
        try:
            product_selectors = [
                ".review-product-name", ".product-name", "[class*='product-name']", 
                "[class*='product_name']", ".product_info", ".product-info"
            ]
            
            for selector in product_selectors:
                try:
                    product_elem = container.find_element(By.CSS_SELECTOR, selector)
                    product_text = product_elem.text.strip()
                    if product_text and '[' in product_text and ']' in product_text:
                        product_name = product_text
                        break
                except:
                    continue
        except:
            pass
        
        # 이미지 정보 수집 (리뷰용만)
        images = container.find_elements(By.TAG_NAME, 'img')
        review_photos = [img.get_attribute("src") for img in images if img.get_attribute("src") and "reviews" in img.get_attribute("src")]
        photo_count = len(review_photos)
        has_photo = photo_count > 0

        # 리뷰 데이터 추출
        review_data = extract_review_data(full_text, debug)
        
        # 이미지 정보 컬럼 추가
        review_data["has_photo"] = has_photo
        review_data["photo_count"] = photo_count

        # 직접 찾은 상품명이 있으면 우선 적용
        if product_name and not review_data["product_name"]:
            review_data["product_name"] = product_name
        
        return review_data
    except Exception as e:
        if debug:
            print(f"리뷰 컨테이너 처리 중 오류: {str(e)}")
        return None

def find_review_containers(driver, debug=False):
    """페이지에서 리뷰 컨테이너 요소 찾기"""
    containers = []
    
    # 1. 직접적인 리뷰 컨테이너 클래스 찾기
    selectors = [
        ".review-container", ".review-item", ".review_item", ".review", 
        "[class*='review-']", "[class*='review_']", 
        ".reviews-list > div", ".reviews-list > li",
        ".crema-reviews > div", ".crema-product-reviews > div"
    ]
    
    for selector in selectors:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            if elements:
                if debug:
                    print(f"{len(elements)}개의 컨테이너 발견 (선택자: {selector})")
                
                for element in elements:
                    try:
                        if element.is_displayed() and len(element.text) > 30:
                            # 리뷰 관련 키워드가 있는지 확인
                            has_review_keywords = any(kw in element.text for kw in ["리뷰", "별점", "★", "님의", "[SW", "[LSW", "[KPC", "2025"])
                            if has_review_keywords:
                                containers.append(element)
                    except:
                        continue
        except:
            continue
    
    # 2. 앵커 요소로부터 상위 컨테이너 찾기
    if len(containers) < 3:  # 충분한 컨테이너를 못 찾은 경우
        anchor_selectors = [
            "//*[contains(text(), '리뷰')]",
            "//*[contains(text(), '★')]",
            "//*[contains(text(), '별점')]",
            "//*[contains(text(), '님의 리뷰입니다')]",
            "//*[contains(text(), '2025')]",
            "//*[contains(text(), 'NEW')]",
            "//*[contains(text(), '네이버페이')]"
        ]
        
        for selector in anchor_selectors:
            try:
                anchors = driver.find_elements(By.XPATH, selector)
                for anchor in anchors:
                    try:
                        # 상위로 올라가며 리뷰 컨테이너 찾기
                        parent = anchor
                        for i in range(5):  # 최대 5단계 상위로 확인
                            parent = parent.find_element(By.XPATH, "./..")
                            if len(parent.text) > 100:  # 충분히 큰 컨테이너
                                has_review_content = any(kw in parent.text for kw in ["[SW", "[LSW", "[KPC", "별점", "★", "네이버페이"])
                                if has_review_content and parent not in containers:
                                    containers.append(parent)
                                    break
                    except:
                        continue
            except:
                continue
    
    # 3. 중복 제거 및 최대 30개만 반환
    unique_containers = []
    seen_contents = set()
    
    for container in containers:
        try:
            content = container.text[:100]  # 내용 일부로 중복 확인
            if content not in seen_contents:
                seen_contents.add(content)
                unique_containers.append(container)
                
                if len(unique_containers) >= 30:  # 최대 30개로 제한
                    break
        except:
            continue
    
    if debug:
        print(f"총 {len(unique_containers)}개의 고유 리뷰 컨테이너 찾음")
    
    return unique_containers

def scroll_and_expand_reviews(driver):
    """페이지 스크롤 및 리뷰 확장"""
    # 페이지 스크롤
    print("페이지 스크롤 시작...")
    
    # 초기 높이 저장
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 스크롤 반복
    max_scrolls = 10  # 최대 스크롤 횟수
    scrolls = 0
    
    while scrolls < max_scrolls:
        # 페이지 아래로 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # 로딩 대기
        random_sleep(1.5, 2.5)
        
        # 새 높이 계산
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # 스크롤 후 새 컨텐츠가 로드되지 않으면 종료
        if new_height == last_height:
            print(f"더 이상 새 컨텐츠가 로드되지 않음 (스크롤 {scrolls+1}회)")
            break
        
        last_height = new_height
        scrolls += 1
        print(f"스크롤 {scrolls}회 완료, 새 높이: {new_height}px")
    
    # '더보기' 버튼 클릭
    try:
        print("리뷰 더보기 버튼 찾는 중...")
        more_buttons = driver.find_elements(By.XPATH, 
            "//*[contains(text(), '더보기') or contains(text(), '리뷰 더보기') or contains(@class, 'more')]")
        
        if more_buttons:
            print(f"{len(more_buttons)}개의 더보기 버튼 발견")
            for button in more_buttons:
                try:
                    if button.is_displayed():
                        driver.execute_script("arguments[0].click();", button)
                        random_sleep(0.5, 1)
                except:
                    continue
    except:
        pass
    
    # 다시 천천히 내려오면서 모든 요소가 렌더링되도록 함
    driver.execute_script("window.scrollTo(0, 0);")
    random_sleep(1, 2)
    
    total_height = driver.execute_script("return document.body.scrollHeight")
    view_height = driver.execute_script("return window.innerHeight")
    
    steps = min(10, int(total_height / view_height) + 1)  # 최대 10단계로 제한
    for step in range(steps):
        driver.execute_script(f"window.scrollTo(0, {step * view_height});")
        random_sleep(0.5, 1)
    
    print("페이지 스크롤 및 리뷰 확장 완료")

def is_duplicate_review(reviews, new_review, threshold=0.7):
    """중복 리뷰인지 확인하는 함수"""
    # 내용이 없으면 판단 불가
    if not new_review.get("content"):
        return False
    
    # 날짜와 제품명이 같은 리뷰 중에서 찾기
    for review in reviews:
        # 날짜가 같고 제품명이 같거나 둘 다 없는 경우
        same_date = review.get("date") == new_review.get("date")
        same_product = (review.get("product_name") == new_review.get("product_name")) or (not review.get("product_name") and not new_review.get("product_name"))
        
        if same_date and same_product:
            # 내용 유사도 검사 (간단한 구현)
            review_content = review.get("content", "")
            new_content = new_review.get("content", "")
            
            # 내용이 너무 짧으면 비교 의미가 없음
            if len(review_content) < 5 or len(new_content) < 5:
                continue
            
            # 같은 글자 수 비율로 유사도 측정
            min_length = min(len(review_content), len(new_content))
            max_check_length = min(50, min_length)  # 최대 50자까지만 비교
            
            # 글자 단위 비교
            common_chars = sum(a == b for a, b in zip(review_content[:max_check_length], new_content[:max_check_length]))
            similarity = common_chars / max_check_length
            
            if similarity > threshold:
                return True
            
            # 네이버페이 구매자 특수 케이스
            review_writer = review.get("writer", "")
            new_review_writer = new_review.get("writer", "")
            
            # None 체크 추가
            if (review_writer is not None and "네이버페이 구매자" in review_writer and 
                new_review_writer is not None and "네이버페이 구매자" in new_review_writer):
                # 발 쏘았습니다--- 같은 패턴
                if "발 쏘았습니다" in review_content and "발 쏘았습니다" in new_content:
                    return True
                
                # 타임스탬프 패턴이 비슷한 경우
                timestamp_review = re.search(r'\d{2}:\d{2}:\d{2}', review_content)
                timestamp_new = re.search(r'\d{2}:\d{2}:\d{2}', new_content)
                if timestamp_review and timestamp_new and timestamp_review.group(0) == timestamp_new.group(0):
                    return True
    
    return False

def generate_dedupe_key(review):
    """중복 판단용 키 생성 함수"""
    date = review.get("date", "")
    product = review.get("product_name", "")
    content = review.get("content", "")
    
    # content 기준으로만 판단 (writer는 마스킹돼있으므로 불안정)
    return f"{date}-{product}-{content[:30].strip()}"

def merge_duplicate_reviews(reviews):
    """중복 리뷰를 병합하는 함수"""
    if not reviews:
        return []
    
    # 리뷰를 날짜-제품명 기준으로 그룹화
    grouped_reviews = {}
    
    for review in reviews:
        key = generate_dedupe_key(review)
        
        if key not in grouped_reviews:
            grouped_reviews[key] = []
        
        # 동일 그룹 내 중복 체크
        is_duplicate = False
        for existing_review in grouped_reviews[key]:
            # 내용 유사도 검사
            if is_duplicate_review([existing_review], review):
                is_duplicate = True
                
                # 기존 리뷰에 내용 추가 (네이버페이 구매자 케이스)
                writer = review.get("writer", "")
                # 수정된 부분: None 체크 추가
                if writer is not None and "네이버페이 구매자" in writer and review.get("content"):
                    existing_content = existing_review.get("content", "")
                    new_content = review.get("content", "")
                    
                    # 새 내용이 기존 내용을 포함하지 않을 경우에만 추가
                    if new_content and new_content not in existing_content:
                        # "발 쏘았습니다" 중복 제거
                        if "발 쏘았습니다" in existing_content and "발 쏘았습니다" in new_content:
                            pass
                        else:
                            combined_content = f"{existing_content} {new_content}".strip()
                            existing_review["content"] = combined_content
                
                # 다른 필드는 비어있으면 채우기
                for field in ["rating", "writer", "height", "weight", "usual_size", "member_level", "size"]:
                    if not existing_review.get(field) and review.get(field):
                        existing_review[field] = review.get(field)
                
                break
        
        # 중복이 아니면 새로 추가
        if not is_duplicate:
            grouped_reviews[key].append(review)
    
    # 그룹별로 정렬하여 1개씩만 선택
    merged_reviews = []
    
    for key, group in grouped_reviews.items():
        if not group:  # 그룹이 비어있으면 건너뛰기
            continue
        
        if len(group) == 1:  # 그룹에 리뷰가 1개만 있으면 그대로 사용
            merged_reviews.append(group[0])
        else:  # 여러 리뷰가 있으면 병합
            # 가장 내용이 긴 리뷰를 기준으로 선택
            best_review = max(group, key=lambda x: len(x.get("content", "")))
            
            # 다른 리뷰에서 부족한 정보 보충
            for review in group:
                if review == best_review:
                    continue
                
                # 각 필드 확인
                for field in ["product_name", "rating", "writer", "date", "height", "weight", "usual_size", "member_level", "size"]:
                    if not best_review.get(field) and review.get(field):
                        best_review[field] = review.get(field)
                
                # 내용 병합 (다른 내용이 있는 경우)
                if review.get("content") and review.get("content") not in best_review.get("content", ""):
                    # 중복된 "발 쏘았습니다" 제거
                    if "발 쏘았습니다" in best_review.get("content", "") and "발 쏘았습니다" in review.get("content", ""):
                        continue
                    
                    # 내용 병합
                    combined_content = f"{best_review.get('content', '')} {review.get('content', '')}".strip()
                    best_review["content"] = combined_content
            
            merged_reviews.append(best_review)
    
    return merged_reviews

def get_reviews_from_page(driver, page_num, debug=False):
    """특정 페이지의 리뷰 수집"""
    url = f"https://review9.cre.ma/swim.co.kr/reviews?page={page_num}&iframe=1"
    print(f"페이지 크롤링 중: {url}")
    
    driver.get(url)
    random_sleep(3, 5)
    
    # 페이지 스크롤 및 리뷰 확장
    scroll_and_expand_reviews(driver)
    
    # 리뷰 컨테이너 찾기
    containers = find_review_containers(driver, debug)
    
    # 각 컨테이너에서 리뷰 데이터 추출
    reviews = []
    for container in containers:
        review_data = process_review_container(container, debug)
        if review_data and review_data["content"]:
            reviews.append(review_data)
    
    # 중복 리뷰 병합 (네이버페이 리뷰 등)
    merged_reviews = merge_duplicate_reviews(reviews)
    
    print(f"페이지 {page_num}에서 총 {len(merged_reviews)}개 리뷰 수집 완료")
    return merged_reviews

def crawl_reviews(start_page=1, end_page=10, output_file=None, debug=False):
    """여러 페이지에 걸쳐 리뷰 수집"""
    all_reviews = []
    
    # 웹드라이버 설정
    driver = setup_driver()
    
    try:
        for page_num in range(start_page, end_page + 1):
            print(f"\n====== 페이지 {page_num} 크롤링 시작 ======")
            page_reviews = get_reviews_from_page(driver, page_num, debug)
            all_reviews.extend(page_reviews)
            
            # 임시 저장 (50페이지마다)
            if page_num % 50 == 0 and all_reviews:
                temp_file = f"swim_reviews_temp_{start_page}_to_{page_num}.csv"
                save_to_csv(all_reviews, temp_file)
                print(f"임시 파일 저장: {temp_file}")
            
            # 다음 페이지 이동 전 잠시 대기
            random_sleep(2, 4)
    finally:
        driver.quit()
        print("웹드라이버 종료")
    
    # 최종 결과 저장
    if all_reviews:
        if not output_file:
            output_file = f"swim_reviews_{start_page}_to_{end_page}.csv"
        save_to_csv(all_reviews, output_file)
        print(f"총 {len(all_reviews)}개 리뷰 수집 완료. 파일 저장: {output_file}")
    else:
        print("수집된 리뷰가 없습니다.")
    
    return all_reviews

def filter_incomplete_reviews(df):
    df_filtered = df[~(
         df["product_name"].isna() |
        (
            df["product_name"].notna() &
            df["date"].isna() &
            df["writer"].isna()
        )
    )]
    return df_filtered

def extract_clean_review_content(raw_text):
    # "리뷰" 이후 텍스트에서 시작
    review_start = re.search(r'리뷰[\s\d,]*', raw_text)
    if review_start:
        start_idx = review_start.end()
        text_after = raw_text[start_idx:].strip()
        
        # "도움돼요"나 "댓글" 이전까지만 가져오기
        end_match = re.search(r'(리뷰 더보기|도움돼요|도움 안돼요|댓글\s*\d+)', text_after)
        if end_match:
            end_idx = end_match.start()
            clean_content = text_after[:end_idx].strip()
        else:
            clean_content = text_after.strip()
        
        # 불필요한 줄바꿈 정리
        clean_content = re.sub(r'\n+', ' ', clean_content).strip()
        return clean_content
    else:
        return raw_text  # fallback

def save_to_csv(reviews, filename=OUTPUT_FILE):
    """수집한 리뷰를 CSV 파일로 저장"""
    if not reviews:
        print("저장할 리뷰가 없습니다.")
        return None
    
    # 데이터프레임 생성
    df = pd.DataFrame(reviews)
    
    # 불완전 리뷰 제거 추가
    df = filter_incomplete_reviews(df)

    # 데이터 정제
    # 빈 값 처리
    df.fillna("", inplace=True)
    
    # 상품명과 내용이 뒤바뀐 경우 수정
    for idx, row in df.iterrows():
        product_name = row['product_name']
        content = row['content']
        
        # 상품명이 없고 내용에 제품 코드 패턴이 있는 경우
        if (not product_name or product_name == "") and content:
            # 내용에서 제품 코드 패턴 찾기
            code_match = re.search(r'\[(?:SW|LSW|KPC|PC|ATA|ASA|A5A|A3A|SC|NES|DMC|LGNS|PCS)[\w\d-]+\]', content)
            if code_match:
                # 제품명 추출
                product_pattern = re.search(r'((?:[\w\s가-힣\(\)]+)?\s*\[(?:SW|LSW|KPC|PC|ATA|ASA|A5A|A3A|SC|NES|DMC|LGNS|PCS)[\w\d-]+\]\s*(?:[\w\s가-힣\(\)]+)?)', content)
                if product_pattern:
                    df.at[idx, 'product_name'] = product_pattern.group(1).strip()
                    # 내용에서 상품명 제거
                    df.at[idx, 'content'] = content.replace(product_pattern.group(1), "").strip()
        
        # 상품명이 있고 내용에도 포함된 경우
        elif product_name and content and product_name in content:
            df.at[idx, 'content'] = content.replace(product_name, "").strip()
    
    # 빈 내용 제거
    df = df[df["content"].str.len() > 0]
    
    # 중요: rating 열을 숫자로 변환
    try:
        # 문자열로 변환 후 숫자로 변환 시도
        df['rating'] = df['rating'].astype(str).str.strip()
        
        # 빈 문자열은 NaN으로 처리
        df.loc[df['rating'] == '', 'rating'] = None
        
        # 숫자로 변환 가능한 값만 변환 (NaN 허용)
        df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
    except Exception as e:
        print(f"별점 데이터 변환 중 오류: {str(e)}")
        # 대체 방법: 숫자가 아닌 값은 NaN으로 변환
        numeric_ratings = []
        for r in df['rating']:
            try:
                numeric_ratings.append(float(r) if r else None)
            except:
                numeric_ratings.append(None)
        df['rating'] = numeric_ratings
    
    # 파일 저장
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"{len(df)}개의 리뷰를 {filename}에 저장했습니다.")
    
    # 데이터 요약
    print("\n리뷰 데이터 요약:")
    print(f"총 제품 수: {df['product_name'].nunique()}")
    
    # 별점 통계 - 예외 처리 추가
    try:
        # NaN 값 제외하고 평균 계산
        mean_rating = df['rating'].dropna().mean()
        if pd.notna(mean_rating):  # NaN이 아닌 경우만 출력
            print(f"평균 별점: {mean_rating:.2f}점")
        else:
            print("평균 별점: 계산 불가 (유효한 별점 없음)")
    except Exception as e:
        print(f"평균 별점 계산 오류: {str(e)}")
    
    # 작성자 통계
    try:
        writer_count = df['writer'].replace('', None).dropna().nunique()
        print(f"작성자 수: {writer_count}명")
    except:
        print("작성자 수: 계산 불가")
    
    # 네이버페이 구매자 리뷰 수
    try:
        naverpay_count = (df['writer'] == "네이버페이 구매자").sum()
        if naverpay_count > 0:
            print(f"네이버페이 구매자 리뷰: {naverpay_count}개")
    except:
        pass
    
    # 수집된 데이터 샘플
    print("\n첫 5개 리뷰 샘플:")
    pd.set_option('display.max_colwidth', 30)  # 컬럼 내용 일부만 표시
    print(df.head().to_string())
    
    return df

def main():
    """메인 함수"""
    print("수영샵 리뷰 크롤링을 시작합니다.")
    
    # 디버깅 모드 활성화 여부
    debug_mode = False
    
    # 크롤링할 페이지 범위 설정
    start_page = 1
    end_page = 3200  # 원하는 페이지 수로 변경
    
    # 크롤링 수행
    reviews = crawl_reviews(
        start_page=start_page, 
        end_page=end_page, 
        output_file=OUTPUT_FILE, 
        debug=debug_mode
    )
    
    print("크롤링 완료!")

if __name__ == "__main__":
    main()

수영샵 리뷰 크롤링을 시작합니다.

====== 페이지 1 크롤링 시작 ======
페이지 크롤링 중: https://review9.cre.ma/swim.co.kr/reviews?page=1&iframe=1
페이지 스크롤 시작...
더 이상 새 컨텐츠가 로드되지 않음 (스크롤 1회)
리뷰 더보기 버튼 찾는 중...
60개의 더보기 버튼 발견
페이지 스크롤 및 리뷰 확장 완료
페이지 1에서 총 27개 리뷰 수집 완료

====== 페이지 2 크롤링 시작 ======
페이지 크롤링 중: https://review9.cre.ma/swim.co.kr/reviews?page=2&iframe=1
페이지 스크롤 시작...
더 이상 새 컨텐츠가 로드되지 않음 (스크롤 1회)
리뷰 더보기 버튼 찾는 중...
60개의 더보기 버튼 발견
페이지 스크롤 및 리뷰 확장 완료
페이지 2에서 총 24개 리뷰 수집 완료

====== 페이지 3 크롤링 시작 ======
페이지 크롤링 중: https://review9.cre.ma/swim.co.kr/reviews?page=3&iframe=1
페이지 스크롤 시작...
더 이상 새 컨텐츠가 로드되지 않음 (스크롤 1회)
리뷰 더보기 버튼 찾는 중...
60개의 더보기 버튼 발견
페이지 스크롤 및 리뷰 확장 완료
페이지 3에서 총 25개 리뷰 수집 완료

====== 페이지 4 크롤링 시작 ======
페이지 크롤링 중: https://review9.cre.ma/swim.co.kr/reviews?page=4&iframe=1
페이지 스크롤 시작...
더 이상 새 컨텐츠가 로드되지 않음 (스크롤 1회)
리뷰 더보기 버튼 찾는 중...
60개의 더보기 버튼 발견
페이지 스크롤 및 리뷰 확장 완료
페이지 4에서 총 25개 리뷰 수집 완료

====== 페이지 5 크롤링 시작 ======
페이지 크롤링 중: https://review9.cre.ma/swim.co.kr/reviews?page=5&iframe=1
페이지 스크롤 시작...
더 이상 새 컨텐츠